In [1]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

## Pipeline A

In [2]:
SERVICE_ACCOUNT_FILE = '../../key/credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

sheet = client.open("2025 Analyze (Like/Dislike)").worksheet("Raw Data [Statistic form] 2024")
data = sheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [3]:
df_clean = df.copy()

# Hapus spasi berlebih
df_clean.columns = df_clean.columns.str.strip()
df_clean.columns = df_clean.columns.str.replace(r'\s+', ' ', regex=True)

# Change to datetime
datetime_col = [
    'Date'
]
for col in datetime_col:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Replacing the empty data with null
df_clean['business'].replace('', 'no data', inplace=True)

# Check is there any NA
if df_clean['business'].isna().any():
    df_clean['business'] = df_clean['business'].fillna('no data')

# Convert to categorycal data type
categorical_cols = [
    'business', 'Manual Check [business]', 'Team/Category', 'Week'
]
for col in categorical_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype('category')

# Convert to numerical data type
numerical_cols = [
    'solved_num','unsolved_num'
]
for col in numerical_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

C:\Users\ozy.prazuganda\AppData\Local\Temp\ipykernel_18632\1173681464.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['business'].replace('', 'no data', inplace=True)


In [4]:
df_clean

,Date,business,code_type,Code,solved_num,unsolved_num,Manual Check [business],Background detail,Team/Category,Week,
0,2024-01-01,no data,模型匹配,200101,17,23,AFI/ASI,chit-chat not identifikasi,No Category,,
1,2024-01-01,AFI,模型匹配,200101,8,36,AFI/ASI,chit-chat not identifikasi,No Category,,
2,2024-01-01,no data,高频FAQ,1344-1365-1427-278,4,6,ASI,Phone balance has not been received,Voucher,,
3,2024-01-01,ASI,模型匹配,1344-1365-1427-278,1,5,ASI,Phone balance has not been received,Voucher,,
4,2024-01-01,ASI,高频FAQ,1344-1365-1427-278,0,10,ASI,Phone balance has not been received,Voucher,,
...,...,...,...,...,...,...,...,...,...,...,...
178957,2025-11-16,4,模型匹配,289-327-484-505,1,0,ASI,Definition of Loan,Loan,W2 Nov 2025,
178958,2025-11-16,ASI,模型匹配,872-919-1014-521,0,1,ASI,Installment Fund Bill has not been updated,Loan,W2 Nov 2025,
178959,2025-11-16,no data,模型匹配,1134-1156-1179-807,0,1,ASI,Changes to missing telephone numbers,Data Update,W2 Nov 2025,
178960,2025-11-16,AFI,模型匹配,1344-1403-1465-722,1,0,ASI,The quota has not been received,Voucher,W2 Nov 2025,


In [5]:
df_clean.to_csv('../../dataset_kula/kula_like_dislike.csv')